In [1]:
import findspark
from pyspark.sql import SparkSession
from pyspark import SparkContext
from pyspark.streaming import StreamingContext

In [2]:
# !pip3.6 install riotwatcher

In [3]:
# Create a local StreamingContext with two working thread and batch interval of 1 second
# sc = SparkContext(, "LOL")

findspark.init()
spark = SparkSession.builder.master("local[*]").getOrCreate()
ssc = StreamingContext(sc, 5)

In [4]:
from riotwatcher import LolWatcher, ApiError
import json
import random
import time

watcher = LolWatcher('RGAPI-bee6aa98-2316-448f-bd43-dac8d90dc7a5')
region = 'EUN1'
random.seed(123)

In [5]:
# used to call multiple games at once
# gamelist=[]
# for i in range (500):
#     gamelist.extend(watcher.spectator.featured_games(region)["gameList"])

In [6]:
# with open('data', 'w') as f:
#     for game in gamelist:
#         f.write(json.dumps(game)+"\n")

In [7]:
games = ssc.socketTextStream("localhost", 9999)
# games=ssc.textFileStream("data")
# games=games.flatMap(lambda x:json.loads(x))

In [8]:
def getTeamMembers(game,team_id):
    participants=game['participants']
    
    [{'summonerName':participant['summonerName'],"championId":participant["championId"]} for participant in game['participants'] if participant['bot']== False and participant['teamId']==100]

In [4]:
import json

json.loads("[{'summonerName': 'King Punisher', 'championId': 50, 'teamId': 100}, {'summonerName': 'Seyroχ', 'championId': 121, 'teamId': 100}, {'summonerName': '2OF US ARE DYING', 'championId': 114, 'teamId': 100}, {'summonerName': 'Nut In Head', 'championId': 39, 'teamId': 100}, {'summonerName': 'Zumei666', 'championId': 40, 'teamId': 100}, {'summonerName': 'Elise Presley', 'championId': 60, 'teamId': 200}, {'summonerName': 'RiverSanzu1', 'championId': 145, 'teamId': 200}, {'summonerName': 'Thę Legend', 'championId': 91, 'teamId': 200}, {'summonerName': 'Nórm', 'championId': 98, 'teamId': 200}, {'summonerName': 'Scaryride', 'championId': 412, 'teamId': 200}]")

JSONDecodeError: Expecting property name enclosed in double quotes: line 1 column 3 (char 2)

In [10]:
ssc.start()
ssc.awaitTermination()

-------------------------------------------
Time: 2020-12-26 10:26:50
-------------------------------------------

-------------------------------------------
Time: 2020-12-26 10:26:55
-------------------------------------------



KeyboardInterrupt: 

In [ ]:
blue=[{'summonerName':participant['summonerName'],"championId":participant["championId"]} for participant in game['participants'] if participant['bot']== False and participant['teamId']==100]
red=[{'summonerName':participant['summonerName'],"championId":participant["championId"]} for participant in game['participants'] if participant['bot']== False and participant['teamId']==200]

In [76]:
def getPlayersInGame(region:str)->tuple:
  watcher.spectator.featured_games(region)
  game=random.choice(gamelist['gameList'])  
  blue=[{'summonerName':participant['summonerName'],"championId":participant["championId"]} for participant in game['participants'] if participant['bot']== False and participant['teamId']==100]
  red=[{'summonerName':participant['summonerName'],"championId":participant["championId"]} for participant in game['participants'] if participant['bot']== False and participant['teamId']==200]
  return(blue,red)

In [77]:
blue,red = getPlayersInGame(region)

In [256]:
def getPlayerScore(player:dict,matches_limit:int=5):
  player_stats={"wins" :0,
                "kills" :0,
                "deaths" :0,
                "assists" :0,
                "largestKillingSpree" :0,
                "largestMultiKill" :0,
                "killingSprees" :0,
                "longestTimeSpentLiving" :0,
                "doubleKills" :0,
                "tripleKills" :0,
                "quadraKills" :0,
                "pentaKills" :0,
                "unrealKills" :0,
                "totalDamageDealt" :0,
                "magicDamageDealt" :0,
                "physicalDamageDealt" :0,
                "trueDamageDealt" :0,
                "largestCriticalStrike" :0,
                "totalDamageDealtToChampions" :0,
                "magicDamageDealtToChampions" :0,
                "physicalDamageDealtToChampions" :0,
                "trueDamageDealtToChampions" :0,
                "totalHeal" :0,
                "totalUnitsHealed" :0,
                "damageSelfMitigated" :0,
                "damageDealtToObjectives" :0,
                "damageDealtToTurrets" :0,
                "visionScore" :0,
                "timeCCingOthers" :0,
                "totalDamageTaken" :0,
                "magicalDamageTaken" :0,
                "physicalDamageTaken" :0,
                "trueDamageTaken" :0,
                "goldEarned" :0,
                "goldSpent" :0,
                "turretKills" :0,
                "inhibitorKills" :0,
                "totalMinionsKilled" :0,
                "neutralMinionsKilled" :0,
                "totalTimeCrowdControlDealt" :0,
                "combatPlayerScore" :0,
                "objectivePlayerScore" :0,
                "totalPlayerScore" :0,
                "totalScoreRank" :0}

  # Player basic game data
  player_name=player["summonerName"]
  player_champ=player["championId"]
  player_data=watcher.summoner.by_name(region,player_name)
  
  # Player Champ data
  account_id=player_data['accountId']
  summoner_id=player_data['id']
  player_champ_data=watcher.champion_mastery.by_summoner_by_champion(region,summoner_id,player_champ)
  
  player_stats["champ_level"]=player_champ_data['championLevel']
  player_stats["champ_points"]=player_champ_data['championPoints']
  player_stats["champ_familiarity"]=(int(round(time.time() * 1000)) - player_champ_data['lastPlayTime'])/1000 #converts it into seconds
  
  #get the matches list the summoner played
  matches_champ=watcher.match.matchlist_by_account(region,encrypted_account_id=accountid,champion=player_champ)
  matches=watcher.match.matchlist_by_account(region,encrypted_account_id=accountid) 

  # aquire the detail stats of all played matches by this player
  matches_data=[]
  for idx,match in enumerate(matches["matches"]):
    if idx==matches_limit: break
    matches_data.append(watcher.match.by_id(region,match["gameId"]))
  matches_number=len(matches_data)

  # loop over all the matches and get the important stats
  for match in matches_data:
    player_id=[participant["participantId"] for participant in match["participantIdentities"] if participant["player"]["summonerId"]==summoner_id]
    assert len(player_id)==1
    player_id=player_id[0]
    participant=[participant for participant in match["participants"] if participant["participantId"]==player_id]
    assert len(participant)==1
    stats = participant[0]["stats"]
    
    if stats["win"]: player_stats["wins"] +=1  #: false
    player_stats["kills"] += stats["kills"]  #: 27
    player_stats["deaths"] += stats["deaths"]  #: 8
    player_stats["assists"] += stats["assists"]  #: 13
    player_stats["largestKillingSpree"] += stats["largestKillingSpree"]  #: 6
    player_stats["largestMultiKill"] += stats["largestMultiKill"]  #: 2
    player_stats["killingSprees"] += stats["killingSprees"]  #: 7
    player_stats["longestTimeSpentLiving"] += stats["longestTimeSpentLiving"]  #: 232
    player_stats["doubleKills"] += stats["doubleKills"]  #: 5
    player_stats["tripleKills"] += stats["tripleKills"]  #: 0
    player_stats["quadraKills"] += stats["quadraKills"]  #: 0
    player_stats["pentaKills"] += stats["pentaKills"]  #: 0
    # player_stats["unrealKills"] += stats["unrealKills"]  #: 0
    player_stats["totalDamageDealt"] += stats["totalDamageDealt"]  #: 145108
    # player_stats["magicDamageDealt"] += stats["magicDamageDealt"]  #: 6740
    # player_stats["physicalDamageDealt"] += stats["physicalDamageDealt"]  #: 136190
    # player_stats["trueDamageDealt"] += stats["trueDamageDealt"]  #: 2176
    # player_stats["largestCriticalStrike"] += stats["largestCriticalStrike"]  #: 847
    # player_stats["totalDamageDealtToChampions"] += stats["totalDamageDealtToChampions"]  #: 55924
    # player_stats["magicDamageDealtToChampions"] += stats["magicDamageDealtToChampions"]  #: 2371
    # player_stats["physicalDamageDealtToChampions"] += stats["physicalDamageDealtToChampions"]  #: 51489
    # player_stats["trueDamageDealtToChampions"] += stats["trueDamageDealtToChampions"]  #: 2063
    # player_stats["totalHeal"] += stats["totalHeal"]  #: 9776
    # player_stats["totalUnitsHealed"] += stats["totalUnitsHealed"]  #: 1
    # player_stats["damageSelfMitigated"] += stats["damageSelfMitigated"]  #: 21852
    player_stats["damageDealtToObjectives"] += stats["damageDealtToObjectives"]  #: 1417
    player_stats["damageDealtToTurrets"] += stats["damageDealtToTurrets"]  #: 1417
    player_stats["visionScore"] += stats["visionScore"]  #: 0
    player_stats["timeCCingOthers"] += stats["timeCCingOthers"]  #: 46
    player_stats["totalDamageTaken"] += stats["totalDamageTaken"]  #: 28124
    player_stats["magicalDamageTaken"] += stats["magicalDamageTaken"]  #: 14883
    player_stats["physicalDamageTaken"] += stats["physicalDamageTaken"]  #: 12785
    player_stats["trueDamageTaken"] += stats["trueDamageTaken"]  #: 456
    player_stats["goldEarned"] += stats["goldEarned"]  #: 19869
    # player_stats["goldSpent"] += stats["goldSpent"]  #: 17100
    player_stats["turretKills"] += stats["turretKills"]  #: 0
    player_stats["inhibitorKills"] += stats["inhibitorKills"]  #: 0
    # player_stats["totalMinionsKilled"] += stats["totalMinionsKilled"]  #: 93
    # player_stats["neutralMinionsKilled"] += stats["neutralMinionsKilled"]  #: 0
    # player_stats["totalTimeCrowdControlDealt"] += stats["totalTimeCrowdControlDealt"]  #: 226
    player_stats["combatPlayerScore"] += stats["combatPlayerScore"]  #: 0
    # player_stats["objectivePlayerScore"] += stats["objectivePlayerScore"]  #: 0
    # player_stats["totalPlayerScore"] += stats["totalPlayerScore"]  #: 0
    # player_stats["totalScoreRank"] += stats["totalScoreRank"]  #: 0

  return player_stats,matches_number

In [255]:
score,matches_number=getPlayerScore(blue[0],50)

In [257]:
score

{'assists': 1101,
 'champ_familiarity': 10283.704,
 'champ_level': 5,
 'champ_points': 64221,
 'combatPlayerScore': 0,
 'damageDealtToObjectives': 69012,
 'damageDealtToTurrets': 69012,
 'damageSelfMitigated': 576146,
 'deaths': 342,
 'doubleKills': 81,
 'goldEarned': 660979,
 'goldSpent': 589705,
 'inhibitorKills': 9,
 'killingSprees': 126,
 'kills': 483,
 'largestCriticalStrike': 11734,
 'largestKillingSpree': 157,
 'largestMultiKill': 97,
 'longestTimeSpentLiving': 12608,
 'magicDamageDealt': 1797006,
 'magicDamageDealtToChampions': 764429,
 'magicalDamageTaken': 485945,
 'neutralMinionsKilled': 0,
 'objectivePlayerScore': 0,
 'pentaKills': 0,
 'physicalDamageDealt': 1720381,
 'physicalDamageDealtToChampions': 559895,
 'physicalDamageTaken': 485504,
 'quadraKills': 3,
 'timeCCingOthers': 1545,
 'totalDamageDealt': 3638687,
 'totalDamageDealtToChampions': 1383935,
 'totalDamageTaken': 1031273,
 'totalHeal': 346157,
 'totalMinionsKilled': 2795,
 'totalPlayerScore': 0,
 'totalScoreRank

In [244]:
player_stats["wins"]/matches_number

0.54

In [236]:
print(json.dumps(player_stats,indent=2,sort_keys=False))

{
  "wins": 27,
  "kills": 483,
  "deaths": 342,
  "assists": 1101,
  "largestKillingSpree": 157,
  "largestMultiKill": 97,
  "killingSprees": 126,
  "longestTimeSpentLiving": 12608,
  "doubleKills": 81,
  "tripleKills": 14,
  "quadraKills": 3,
  "pentaKills": 0,
  "unrealKills": 0,
  "totalDamageDealt": 3638687,
  "magicDamageDealt": 1797006,
  "physicalDamageDealt": 1720381,
  "trueDamageDealt": 121256,
  "largestCriticalStrike": 11734,
  "totalDamageDealtToChampions": 1383935,
  "magicDamageDealtToChampions": 764429,
  "physicalDamageDealtToChampions": 559895,
  "trueDamageDealtToChampions": 59565,
  "totalHeal": 346157,
  "totalUnitsHealed": 170,
  "damageSelfMitigated": 576146,
  "damageDealtToObjectives": 69012,
  "damageDealtToTurrets": 69012,
  "visionScore": 0,
  "timeCCingOthers": 1545,
  "totalDamageTaken": 1031273,
  "magicalDamageTaken": 485945,
  "physicalDamageTaken": 485504,
  "trueDamageTaken": 59781,
  "goldEarned": 660979,
  "goldSpent": 589705,
  "turretKills": 29,
